*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-03-08 15:34:03--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  75.4MB/s    in 0.3s    

2022-03-08 15:34:04 (75.4 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

# Data preprocessing
To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

In [4]:
# users with at least 200 ratings
users_keep = df_ratings.user.value_counts()
users_keep = users_keep[users_keep>=200].index

# books with at least 100 ratings
books_keep = df_ratings.isbn.value_counts()
books_keep = books_keep[books_keep>=100].index

print(f'Users with at least 200 ratings: {len(books_keep)}',
      f'Books with at least 100 ratings: {len(users_keep)}',
      sep='\n')

Users with at least 200 ratings: 731
Books with at least 100 ratings: 905


In [5]:
# keep only ratings where both conditions are true
ratings = df_ratings[df_ratings.isbn.isin(books_keep) & df_ratings.user.isin(users_keep)]
print(f'Before:\t{df_ratings.shape[0]}',
      f'After:\t{ratings.shape[0]}',
      sep='\n')

Before:	1149780
After:	49781


In [6]:
# ISSUE: not all isbn in ratings are enlisted in df_books
ratings.isbn.isin(df_books.isbn).value_counts()

True     49517
False      264
Name: isbn, dtype: int64

In [7]:
# remove books not in df_books
ratings = ratings[ratings.isbn.isin(df_books.isbn)]
print('N of ratings:', ratings.shape[0])

N of ratings: 49517


In [8]:
# creating dictionary such that {isbn: title}
isbn_to_book = df_books.set_index('isbn')['title'].to_dict()

# adding column title
ratings['title'] = [isbn_to_book[isbn] for isbn in ratings.isbn]
ratings.head()

,user,isbn,rating,title
1456,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
1469,277427,0060930535,0.0,The Poisonwood Bible: A Novel
1471,277427,0060934417,0.0,Bel Canto: A Novel
1474,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...
1484,277427,0140067477,0.0,The Tao of Pooh


In [9]:
print(f'N. of books: {ratings.title.nunique()}',
      f'N. of users: {ratings.user.nunique()}',
      sep='\n')

N. of books: 673
N. of users: 888


In [10]:
X = pd.pivot_table(ratings, values='rating', index='title', columns='user', fill_value=0).sort_index()
pd.set_option('display.max_columns', 12)
X

user,254,2276,2766,2977,3363,4017,...,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,
1984,9,0,0,0,0,0,...,0,0,0,0,0,0
1st to Die: A Novel,0,0,0,0,0,0,...,0,0,0,0,0,0
2nd Chance,0,10,0,0,0,0,...,0,0,0,0,0,0
4 Blondes,0,0,0,0,0,0,...,0,0,0,0,0,0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0,0,0,0,0,0,...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0,0,0,0,0,0,...,0,0,0,0,0,0
Year of Wonders,0,0,0,7,0,0,...,0,0,0,0,0,0
You Belong To Me,0,0,0,0,0,0,...,0,0,0,0,0,0


# Model

In [11]:
model = NearestNeighbors(algorithm='brute',
                         metric='cosine')
model.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
# function to return recommended books - this will be tested
def get_recommends(title = "", K = 5):
    ratings = X.loc[title]
    distances, books = model.kneighbors([ratings], n_neighbors=K)
    distances, books = distances.flatten()[1:], books.flatten()[1:] # first neighbor == title

    # list of lists, where inner lists are [book title, distance]
    recommended = []
    for i, book in enumerate(books):
        recommended.append([X.index[book],
                            distances[i]])

    return [title, sorted(recommended, key=lambda b: b[1], reverse=True)]

In [13]:
# testing example in description
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
get_recommends(title, K=6)

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.7939835419270879],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448657003312193],
  ['Interview with the Vampire', 0.7345068863988313],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   0.5376338446489461],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178411864186413]]]

# Testing
Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [14]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
